In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [4]:
import mne

In [ ]:
# bids_root = '/Users/noehsueh/UBA/Brainhack/Project/dataset'
# print(make_report(bids_root))
# datatype = 'eeg'
# suffix = 'eeg'
# task = 'ImaginedEmotion' 
# subject = '01'
# bids_path = BIDSPath(subject=subject, task=task, 
#                      suffix=suffix, datatype=datatype, 
#                     root=bids_root)
# display(bids_path)
# raw = read_raw_bids(bids_path=bids_path, verbose = False)
# raw.drop_channels(['EXG1', 'EXG2', 'EXG3', 'EXG6']) # drop external channels

In [105]:
# # definir tiempo minimo
# tmin = -0.2
# # definir tiempo maximo
# tmax = 0.8
# # definir baseline
# baseline = (-0.2,0)

## Creación de pipelines para para extraer datos

In [29]:
import mne 
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [92]:
# raw_path = '/Users/noehsueh/UBA/Brainhack/Project/dataset/sub-01/eeg/sub-01_task-ImaginedEmotion_eeg.set'
# raw = mne.io.read_raw_eeglab(raw_path, preload=True) 
# events, event_id = mne.events_from_annotations(raw = raw)
# exclude = [1,2,3,4,5,11,13,21,22,23,24,25,26,27]
# events_included = mne.pick_events(events = events, exclude = exclude) # this wont work
# event_id_included  = {
#     'anger': 6, 
#     'awe': 7, 
#     'compassion': 8, 
#     'content': 9, 
#     'disgust': 10, 
#     'excite': 12, 
#     'fear': 14, 
#     'frustration': 15, 
#     'grief': 16, 
#     'happy': 17, 
#     'jealousy': 18, 
#     'joy': 19, 
#     'love': 20,  
#     'relief': 28, 
#     'sad': 29
# }

# tmin = -0.2
# tmax = 0.8
# baseline = (-0.2,0)

# epochs = mne.Epochs(raw_avg_ref, events = events_included, event_id = event_id_included, 
#                     tmin=tmin, tmax=tmax, baseline=baseline)
# df = epochs.to_data_frame(long_format=True)

In [219]:
raw_path = '/Users/noehsueh/UBA/Brainhack/Project/dataset/sub-01/eeg/sub-01_task-ImaginedEmotion_eeg.set'
raw = mne.io.read_raw_eeglab(raw_path, preload=True) 
raw.resample(128) 
events_included, event_id_included  = mne.events_from_annotations(raw = raw, 
                                                                  event_id = None, regexp='|'.join(selected_annotations))
# tmin = -0.2
# tmax = 180
epochs = mne.Epochs(raw, 
                    events = events_included, 
                    event_id = event_id_included, 
                    tmin=-0.2, tmax=180,baseline=(-0.2,0))
df = epochs.to_data_frame(long_format=True)


Reading /Users/noehsueh/UBA/Brainhack/Project/dataset/sub-01/eeg/sub-01_task-ImaginedEmotion_eeg.fdt
Reading 0 ... 1154303  =      0.000 ...  4508.996 secs...


/var/folders/7z/chvf52ys14n76739wydt9_5w0000gn/T/ipykernel_7679/238599111.py:2: RuntimeWarning: Estimated head radius (0.0 cm) is below the 3rd percentile for infant head size. Check if the montage_units argument is correct (the default is "mm", but your channel positions may be in different units).
  raw = mne.io.read_raw_eeglab(raw_path, preload=True)


Used Annotations descriptions: ['anger', 'awe', 'compassion', 'content', 'disgust', 'excite', 'fear', 'frustration', 'grief', 'happy', 'jealousy', 'joy', 'love', 'relief', 'sad']
Not setting metadata
15 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 15 events and 23067 original time points ...
0 bad epochs dropped
Converting "condition" to "category"...
Converting "epoch" to "category"...
Converting "channel" to "category"...
Converting "ch_type" to "category"...


In [222]:
epochs.get_data().shape

Using data from preloaded Raw for 15 events and 23067 original time points ...


(15, 224, 23067)

# Convertir dataset en X, y 

In [201]:
global positive_emotions 
positive_emotions = ['awe', 'joy', 'happy', 'love', 'compassion', 'content', 'relief', 'excite']

In [206]:
X = epochs.get_data()*1000  # EEG signals: n_epochs, n_eeg_channels, n_times
y_labels = epochs.events[:, 2]  # target
event_id_reverse = {v: k for k, v in event_id_included.items()}  # Reverse the dictionary for mapping event IDs to descriptions
y_labels = [event_id_reverse[label] for label in epochs.events[:, -1]]
y = [1 if label in positive_emotions else 0 for label in y_labels]
y

Using data from preloaded Raw for 15 events and 129 original time points ...


[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

In [207]:
X.shape # 15 emotions, 224 channels, 129 sample rate

(15, 224, 129)

In [141]:
kernels, chans, samples = 1, 64, 129

In [154]:
from EEGModels import EEGNet
from tensorflow.keras import utils as np_utils
tf.config.run_functions_eagerly

In [213]:
y_labels

['awe',
 'frustration',
 'joy',
 'anger',
 'happy',
 'sad',
 'love',
 'grief',
 'compassion',
 'fear',
 'content',
 'jealousy',
 'relief',
 'disgust',
 'excite']

In [211]:
X_train = np.array(X[:,:64])
y_train = np.array(y)
y_train = np_utils.to_categorical((y_train-1)+1)
y_train

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [214]:
X_train = X_train.reshape(X_train.shape[0], chans, samples, kernels)
X_train.shape

(15, 64, 129, 1)

In [215]:
# configure the EEGNet-8,2,16 model with kernel length of 64 samples (other 
# model configurations may do better, but this is a good starting point)
model = EEGNet(nb_classes = 2, Chans = chans, Samples = samples, 
               dropoutRate = 0.5, kernLength = 64, F1 = 8, D = 2, F2 = 16, 
               dropoutType = 'Dropout')
# compile the model and set the optimizers
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics = ['accuracy'])

In [216]:
fittedModel = model.fit(X_train, y_train, batch_size = 2, epochs = 20, 
                        verbose = 2)

Epoch 1/20


/Users/noehsueh/miniconda3/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


8/8 - 0s - loss: 0.7063 - accuracy: 0.4000 - 456ms/epoch - 57ms/step
Epoch 2/20
8/8 - 0s - loss: 0.6518 - accuracy: 0.6000 - 451ms/epoch - 56ms/step
Epoch 3/20
8/8 - 0s - loss: 0.6653 - accuracy: 0.6000 - 449ms/epoch - 56ms/step
Epoch 4/20
8/8 - 0s - loss: 0.6594 - accuracy: 0.6667 - 417ms/epoch - 52ms/step
Epoch 5/20
8/8 - 0s - loss: 0.6554 - accuracy: 0.7333 - 412ms/epoch - 51ms/step
Epoch 6/20
8/8 - 0s - loss: 0.6525 - accuracy: 0.6667 - 416ms/epoch - 52ms/step
Epoch 7/20
8/8 - 0s - loss: 0.6619 - accuracy: 0.6667 - 419ms/epoch - 52ms/step
Epoch 8/20
8/8 - 0s - loss: 0.6300 - accuracy: 0.7333 - 415ms/epoch - 52ms/step
Epoch 9/20
8/8 - 0s - loss: 0.6132 - accuracy: 0.8000 - 414ms/epoch - 52ms/step
Epoch 10/20
8/8 - 0s - loss: 0.5763 - accuracy: 0.6667 - 411ms/epoch - 51ms/step
Epoch 11/20
8/8 - 0s - loss: 0.5622 - accuracy: 0.8000 - 416ms/epoch - 52ms/step
Epoch 12/20
8/8 - 0s - loss: 0.6297 - accuracy: 0.6667 - 431ms/epoch - 54ms/step
Epoch 13/20
8/8 - 0s - loss: 0.5758 - accuracy: 

In [196]:
y_labels

['awe',
 'frustration',
 'joy',
 'anger',
 'happy',
 'sad',
 'love',
 'grief',
 'compassion',
 'fear',
 'content',
 'jealousy',
 'relief',
 'disgust',
 'excite']